## Deliverable 2. Create a Customer Travel Destinations Map.

In [58]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,2022-06-11 18:22:31,71.9667,102.5000,46.87,64,45,9.62,scattered clouds
1,1,Mahebourg,MU,2022-06-11 18:23:03,-20.4081,57.7000,73.69,73,75,10.36,broken clouds
2,2,Cape Town,ZA,2022-06-11 18:20:08,-33.9258,18.4232,70.12,48,0,6.91,clear sky
3,3,Longyearbyen,SJ,2022-06-11 18:19:57,78.2186,15.6401,39.04,70,75,9.22,broken clouds
4,4,Castro,BR,2022-06-11 18:23:04,-24.7911,-50.0119,53.29,59,100,5.41,overcast clouds


In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 68
What is the maximum temperature you would like for your trip? 85


In [50]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mahebourg,MU,2022-06-11 18:23:03,-20.4081,57.7000,73.69,73,75,10.36,broken clouds
2,2,Cape Town,ZA,2022-06-11 18:20:08,-33.9258,18.4232,70.12,48,0,6.91,clear sky
5,5,Albany,US,2022-06-11 18:23:05,42.6001,-73.9662,77.77,50,100,1.01,overcast clouds
8,8,Kapaa,US,2022-06-11 18:23:06,22.0752,-159.3190,78.78,83,75,12.66,broken clouds
10,10,Rikitea,PF,2022-06-11 18:23:08,-23.1203,-134.9692,74.25,80,30,13.49,light rain
12,12,Qui Nhon,VN,2022-06-11 18:23:09,13.7667,109.2333,78.96,84,66,3.42,broken clouds
16,16,Itarema,BR,2022-06-11 18:23:12,-2.9248,-39.9167,82.18,72,66,11.30,broken clouds
21,21,Hilo,US,2022-06-11 18:20:00,19.7297,-155.0900,80.28,68,75,3.00,broken clouds
25,25,Teguise,ES,2022-06-11 18:23:17,29.0605,-13.5640,70.02,73,20,19.57,few clouds
26,26,Gizo,SB,2022-06-11 18:23:18,-8.1030,156.8419,80.91,75,4,4.59,clear sky


In [67]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                287
City                   287
Country                287
Date                   287
Lat                    287
Lng                    287
Max Temp               287
Humidity               287
Cloudiness             287
Wind Speed             287
Current Description    287
dtype: int64

In [60]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,73.69,broken clouds,-20.4081,57.7000,
2,Cape Town,ZA,70.12,clear sky,-33.9258,18.4232,
5,Albany,US,77.77,overcast clouds,42.6001,-73.9662,
8,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,
10,Rikitea,PF,74.25,light rain,-23.1203,-134.9692,
12,Qui Nhon,VN,78.96,broken clouds,13.7667,109.2333,
16,Itarema,BR,82.18,broken clouds,-2.9248,-39.9167,
21,Hilo,US,80.28,broken clouds,19.7297,-155.0900,
25,Teguise,ES,70.02,few clouds,29.0605,-13.5640,
26,Gizo,SB,80.91,clear sky,-8.1030,156.8419,


In [61]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,73.69,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
2,Cape Town,ZA,70.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
5,Albany,US,77.77,overcast clouds,42.6001,-73.9662,
8,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Rikitea,PF,74.25,light rain,-23.1203,-134.9692,People ThankYou
12,Qui Nhon,VN,78.96,broken clouds,13.7667,109.2333,Khách sạn Hải Âu Quy Nhơn - Seagull Hotel
16,Itarema,BR,82.18,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
21,Hilo,US,80.28,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
25,Teguise,ES,70.02,few clouds,29.0605,-13.5640,B&B La Casona de Nazaret
26,Gizo,SB,80.91,clear sky,-8.1030,156.8419,Imagination Island


In [83]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('',np.nan, inplace=True)
hotel_df.dropna().head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,73.69,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
2,Cape Town,ZA,70.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
8,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Rikitea,PF,74.25,light rain,-23.1203,-134.9692,People ThankYou
12,Qui Nhon,VN,78.96,broken clouds,13.7667,109.2333,Khách sạn Hải Âu Quy Nhơn - Seagull Hotel


In [82]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [77]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [84]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5
                )
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))